***GENERATED CODE FOR loanpayoffprediction PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***SELECTED MODEL PREDICTION FUNCTION***

In [ ]:
import json
import sklearn.metrics
import mlflow.sklearn
import pyspark
import mlflow
import sklearn
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'xqd12160159', 'max': 'xqd91160291', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_ID'}, {'feature_label': 'Loan_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Loan_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan_status', 'transformation_label': 'String Indexer'}], 'feature': 'loan_status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'COLLECTION', 'max': 'PAIDOFF', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan_status'}, {'feature_label': 'loan_status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan_status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'effective_date', 'transformation_label': 'String Indexer'}], 'feature': 'effective_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '9/10/2016', 'max': '9/9/2016', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'effective_date'}, {'feature_label': 'effective_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('effective_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'due_date', 'transformation_label': 'String Indexer'}], 'feature': 'due_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '10/10/2016', 'max': '9/28/2016', 'missing': '0', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'due_date'}, {'feature_label': 'due_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('due_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'paid_off_time', 'transformation_label': 'String Indexer'}], 'feature': 'paid_off_time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '400', 'mean': '', 'stddev': '', 'min': '10/1/2016 10:18', 'max': '9/9/2016 13:45', 'missing': '100', 'distinct': '320'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'paid_off_time'}, {'feature_label': 'paid_off_time', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('paid_off_time')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bechalor', 'max': 'college', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


class ModelPredictionClass:

    def __init__(self, sparkDf, runID, label, mType, model_path, subtype):
        self.sparkDf = sparkDf
        self.runID = runID
        self.y = label
        self.yTest = []
        self.yPredicted = []
        self.mType = mType
        self.model_path = model_path
        self.subtype = subtype

    def getmetricsRegression(self):
        scoreMetrics = {}
        scoreMetrics["R2"] = round(
            sklearn.metrics.r2_score(self.yTest, self.yPredicted), 1)
        scoreMetrics["Mean_Squared_Error"] = round(
            sklearn.metrics.mean_squared_error(self.yTest, self.yPredicted), 1)

        return scoreMetrics

    def getmetricsClassification(self):
        scoreMetrics = {}
        self.yPredicted = [int(x) for x in self.yPredicted]
        self.yTest = [int(x) for x in self.yTest]
        scoreMetrics["Accuracy"] = round(
            (100 * sklearn.metrics.accuracy_score(y_true=self.yTest, y_pred=self.yPredicted)), 1)
        scoreMetrics["F1"] = round(
            (100 * sklearn.metrics.f1_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        scoreMetrics["Precision"] = round((
            100 * sklearn.metrics.precision_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")),
            1)
        scoreMetrics["Recall"] = round((
            100 * sklearn.metrics.recall_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        return scoreMetrics

    def getPrediction(self):
        scoreMetrics = {}
        probabilities = {}
        self.sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        self.sparkDf.toPandas()
        data = self.sparkDf.toPandas()
        complete_model_path = "runs:/" + self.runID + "/" + self.model_path
        self.yTest = list(data[self.y])
        data = data.drop(self.y, axis=1).values
        # model get
        sklearn_model = mlflow.sklearn.load_model(
            model_uri=complete_model_path)
        self.yPredicted = sklearn_model.predict(data).tolist()
        if "classification" == self.mType:
            self.yPredicted = [int(i) for i in self.yPredicted]
            scoreMetrics['result'] = self.getmetricsClassification()

            if self.subtype == "naivebayes" or self.subtype == "multinomialNB" or self.subtype == "GaussianNB" or self.subtype == "ComplementNB" or self.subtype == "BernoulliNB" or self.subtype == "CategoricalNB":
                classes = sklearn_model.classes_.tolist()
                scoreMetrics['classes'] = classes
                for i in range(len(classes)):
                    yproba = sklearn_model.predict_proba(data)[:, i].tolist()
                    my_list_rounded = list(
                        map(lambda x: round(x, ndigits=4), yproba))
                    col_name = "Probability_" + str(classes[i])
                    scoreMetrics[col_name] = my_list_rounded
                    probabilities[col_name] = my_list_rounded
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
            else:
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
        elif "regression" == self.mType:
            scoreMetrics['result'] = self.getmetricsRegression()
            scoreMetrics['yTest'] = self.yTest
            scoreMetrics['yPredicted'] = self.yPredicted
            return scoreMetrics

        elif "clustering" == self.mType:
            self.yPredicted = sklearn_model.predict(data).tolist()
            data['prediction'] = self.yPredicted
            return data


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run loanpayoffpredictionHooks.ipynb
try:
	#sourcePreExecutionHook()

	loanpaymenttet = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1695736358591_Loan_payment_test.csv', 'filename': 'Loan_payment_test.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Finance/Loan PayOff/Loan_payment_test.csv', 'viewFileName': 'Loan_payment_test.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICTING GIVEN DATA WITH SELECTED MODEL***

In [ ]:
#%run loanpayoffpredictionHooks.ipynb
try:
	#prediction-pipelinePreExecutionHook()

	loanpaymenttet = TransformationMain.run(loanpaymenttet,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Loan_ID", "transformation_label": "String Indexer"}], "feature": "Loan_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "xqd12160159", "max": "xqd91160291", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_ID"}, {"transformationsData": [{"feature_label": "loan_status", "transformation_label": "String Indexer"}], "feature": "loan_status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "COLLECTION", "max": "PAIDOFF", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Principal", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "943.2", "stddev": "115.24", "min": "300", "max": "1000", "missing": "0"}, "updatedLabel": "Principal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "terms", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "22.82", "stddev": "8.0", "min": "7", "max": "30", "missing": "0"}, "updatedLabel": "terms"}, {"transformationsData": [{"feature_label": "effective_date", "transformation_label": "String Indexer"}], "feature": "effective_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "9/10/2016", "max": "9/9/2016", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "effective_date"}, {"transformationsData": [{"feature_label": "due_date", "transformation_label": "String Indexer"}], "feature": "due_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "10/10/2016", "max": "9/28/2016", "missing": "0", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "due_date"}, {"transformationsData": [{"feature_label": "paid_off_time", "transformation_label": "String Indexer"}], "feature": "paid_off_time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "400", "mean": "", "stddev": "", "min": "10/1/2016 10:18", "max": "9/9/2016 13:45", "missing": "100", "distinct": "320"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "paid_off_time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "past_due_days", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "200", "mean": "36.01", "stddev": "29.38", "min": "1", "max": "76", "missing": "300"}, "updatedLabel": "past_due_days"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "31.12", "stddev": "6.08", "min": "18", "max": "51", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bechalor", "max": "college", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}]}))

MPRe =  ModelPredictionClass(loanpaymenttet,eb56e26ea79147da9ace6084c6e3a904, loan_status_stringindexer, classification, platformmodelsTPOT_loanpayoffclassification_20231114-045934_best_classifier_model, Automl):
responseMetrics = MPRe.getPrediction()
#prediction-pipelinePostExecutionHook(responseMetrics)


except Exception as ex: 
	logging.error(ex)
spark.stop()


***WRITING DATAFRAME***

In [ ]:
#%run loanpayoffpredictionHooks.ipynb
try:
	#sinkPreExecutionHook()

	loanpayoff = NumtraConnector.fetch(spark, "{'samplefile': None, 'samplecount': None, 'originalcount': None, 'orignalKey': None, 'pathOnly': '/Finance/Loan PayOff', 'project_id': '6512e1ceb91dbaa301dd0d6b', 'parent_id': '6512e1ceb91dbaa301dd0d6b', 'original_schema': [], 'actual_schema': [], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'LoanPayOffPrediction.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-81f554cc-6f1f-4f00-9166-59bdb26669b4-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
